# FLOWCELL | OPERATION

[2024-05-09] Notez ici le suivi des opérations en commençant par modifier en titre les noms de la _flowcell_ (_e.g._.: "230711_A00516_0433_BHKVMJDMXY", où `${FC_SHORT}` serait dans ce cas "A00516_0433") et de l'exérience ("Seq_S2_PRAG_20230711"). Ces informations sont normalement communiqués dans un courriel du laboratoire CQGC, mais peuvent aussi être récupérées depuis BSSH (sous l'onglet "_Runs_"). **_N.B._** Le format `ipynb` (Jupyter Notebook) est utilisé car du code peut y être exécuté, mais un simple format `txt` ou `md` peut aussi bien servir.

2. Créer les cas sur Emedgene et lancer les analyses grâce au fichier CSV.
3. (**TODO**) Ajouter les participants _via_ l'API
4. Collecter les métriques des analyses Emedgene `python /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/emg_collect_samples_metrics.py ${FC_SHORT}`
5. Archiver les résultats


In [ ]:
# 0. Mise en place de l'environnement de travail. Il suffit seulement de renseigner 
# la valeur de FC et de copier-coller les commandes dans votre shell sur spxp-app02
# ssh ${USER}@10.128.80.26

export FC="" # Renseigner le nom de la FLOWCELL 
a=($(echo ${FC} | tr '_' '\n'))
export FC_SHORT="${a[1]}_${a[2]}"
export BASEDIR="/mnt/spxp-app02/staging/hiseq_raw/${a[1]}"
export WORKDIR="/mnt/spxp-app02/staging2/dragen"

cd ${BASEDIR}


In [ ]:
# Lister les échantillons et téléverser les FASTQs correspondants sur BSSH

fastqdir="${BASEDIR}/${FC}/Analysis/1/Data/DragenGermline/fastq"
cd ${fastqdir}

project="PRAGMatIQ_CHUQ"
projectId=$( bs -c cac1 project list --filter-term ${project} --terse )
for sample in {26238..26243} {26367..26369}; do
    fastqs=$( ls ${sample}_*_001.fastq.gz )
    bs -c cac1 dataset upload --project ${projectId} --biosample-name ${sample} ${fastqs}
done

project="PRAGMatIQ_CHUSJ"
projectId=$( bs -c cac1 project list --filter-term ${project} --terse )
for sample in {26265..26270}; do
    fastqs=$( ls ${sample}_*_001.fastq.gz )
    bs -c cac1 dataset upload --project ${projectId} --biosample-name ${sample} ${fastqs}
done

project="PRAGMatIQ_CUSM"
projectId=$( bs -c cac1 project list --filter-term ${project} --terse )
for sample in {26131..26133}; do
    fastqs=$( ls ${sample}_*_001.fastq.gz )
    bs -c cac1 dataset upload --project ${projectId} --biosample-name ${sample} ${fastqs}
done


In [ ]:
# 1. Récupérer les informations sur les familles dans Nanuq. Le script suivant génère le fichier CSV d'entrée `emg_batch_manifest.csv`, avec les chemins d'accès aux FASTQs sur BSSH

python /staging2/soft/CQGC-utils/Analysis.pipeline_illumina/emg_make_batch_from_nanuq.py ${FC_SHORT}`



In [ ]:
# Créer le rapport avec les métriques

cd D:\HSJ\Projects\PRAGMatIQ\Runs\LH00336_0041
python D:\HSJ\Workspace\cqgc-dev\pipeline_illumina\emg_collect_samples_metrics.py LH00336_0041 

In [ ]:
# Archiver les résultats

ssh hien@narval.calculquebec.ca

samples=()
total_samples=${#samples[@]}

cd ${HOME}/projects/ctb-rallard/COMMUN/PRAGMatIQ-EMG
cp archive_PRAGMatIQ.log archive_PRAGMatIQ.log0

count=1
for sample in ${samples[@]}; do
    echo "Processing sample ${count}/${total_samples}, ${sample}"
    aws s3 --profile emedgene cp s3://cac1-prodca-emg-auto-results/CHU_Sainte_Justine/${sample}/ ./archives/${sample} --recursive
    echo ${sample} $( date +'%Y-%m-%d %T' ) >> archive_PRAGMatIQ.log
    ((count++))
done